<a href="https://colab.research.google.com/github/ikoojos/Algorithm-Debt-Research/blob/master/Random_Forest_TFIDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
from google.colab import drive
from itertools import product
from sklearn.ensemble import RandomForestClassifier
import importlib

drive.mount('/content/drive')
sys.path.append('/content/drive/My Drive/AD Final Experiments')

from preprocessing import preprocess_data
from splitting import split_data
from utils import *
from evaluate_model import evaluate_best_model
from rf_tuning import hyperparameter_tuning_rf


for module in ['preprocessing', 'splitting', 'utils', 'evaluate_model', 'rf_tuning']:
    importlib.reload(sys.modules[module])

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/My Drive/AD Identification using SATD/liu_datset_processed.csv'
data = preprocess_data(file_path)
X_train_final, X_val, X_test, y_train_final, y_val, y_test = split_data(data)

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 100],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2]
}

vectorizer = TfidfVectorizer()
best_model, best_params, best_score = hyperparameter_tuning_rf(X_train_final, y_train_final, X_val, y_val, param_grid, vectorizer)
evaluate_best_model(best_model, best_params, best_score, X_test, y_test)

Best parameters found: {'n_estimators': 200, 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 1}
Validation set accuracy: 0.8829581993569132

Test Confusion Matrix:
[[  53    0    0   98    0    1    0   48]
 [   0   31    0   38    0    1    1   18]
 [   0    0   36   73    0    1    3   22]
 [   0    0    2 1979    0   22    3  200]
 [   0    0    0   13    6    0    0    4]
 [   0    0    2  165    0  199    1   20]
 [   0    0    0   58    0    1   69   15]
 [   2    1    1   74    1    0    0 4513]]

Test Classification Report:
                        precision    recall  f1-score   support

             ALGORITHM       0.96      0.27      0.42       200
         COMPATIBILITY       0.97      0.35      0.51        89
                DEFECT       0.88      0.27      0.41       135
                DESIGN       0.79      0.90      0.84      2206
         DOCUMENTATION       0.86      0.26      0.40        23
        IMPLEMENTATION       0.88      0.51      0.65       38